In [2]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\feder\Documents\GitHub\relgan\src\jupyter_tests


In [3]:
os.chdir("..")
cwd = os.getcwd()
print(cwd)
assert cwd.endswith("src")

C:\Users\feder\Documents\GitHub\relgan\src


# Data Analysis on the Amazon Attribute dataset

In [4]:
import pandas as pd
import numpy as np
from os.path import join
import nltk
from tqdm import tqdm
from utils.text_process import get_word_list, get_dict, text_to_code
from path_resolution import resources_path
from utils.static_file_manage import write_json

In [5]:
data_dir = resources_path("data", "Amazon_Attribute")

### Dataset import

In [6]:
data = pd.read_csv(join(data_dir, 'full_datasets', 'train.txt'), sep="\t", header=None)
data.columns = ["user_id", "product_id", "rating", "review"]

In [7]:
data.head()

,user_id,product_id,rating,review
0,17362,41609,5.0,better than most -
1,19615,3908,5.0,i enjoy reading so good reading123 <num>
2,1996,33396,4.0,book bub . m . l .
3,7960,33396,4.0,"good book , . z s a"
4,5458,33595,4.0,you will be interested throughout your read.a


#### Compute product_ids and user_ids to remove because they occur too few times

In [8]:
product_count = data['product_id'].value_counts()
product_id_keep = product_count[product_count.apply(lambda x: x>15)].index

user_count = data['user_id'].value_counts()
user_id_keep = user_count[user_count.apply(lambda x: x>6)].index

In [9]:
def remove_low_index(df, user_id_keep, product_id_keep):
    df = df[df['user_id'].isin(user_id_keep)]
    return df[df.product_id.isin(product_id_keep)]

def add_tokenized_column():
    pass

In [10]:
token_list = []
dataframes = []
file_names = ['train.txt', 'test.txt', 'dev.txt']
for file_name in file_names:
    data = pd.read_csv(join(data_dir, 'full_datasets',file_name), sep="\t", header=None)
    data.columns = ["user_id", "product_id", "rating", "review"]
    data = remove_low_index(data, user_id_keep, product_id_keep)
    for text in tqdm(data.review):
        text = nltk.word_tokenize(text.lower())
        token_list.append(text)
    dataframes.append(data)
    
word_set = get_word_list(token_list)
[word_index_dict, index_word_dict] = get_dict(word_set)
sequence_len = len(max(token_list, key=len))
    
    
# data.to_csv(join(data_dir, file_name[:-4] + '.csv'), sep='\t')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22644/22644 [00:07<00:00, 2905.42it/s]


In [33]:
df = dataframes[0]
sorted_users = np.sort(df.user_id.unique())
user_remap = dict(zip(sorted_users, range(len(sorted_users))))
sorted_products = np.sort(df.product_id.unique())
product_remap = dict(zip(sorted_products, range(len(sorted_products))))

In [39]:
len(sorted_products)

5055

In [28]:
df.head()

,user_id,product_id,rating,review,tokenized_text
6,9200,77458,5.0,love this book too have read it again,0 1 2 3 4 5 6 7 80611 80611 80611 80611 80611 ...
14,10646,49714,5.0,i really enjoyed this book . : ),8 9 10 1 2 11 12 13 80611 80611 80611 80611 80...
37,7995,11002,5.0,fast paced and a load of action .,14 15 16 17 18 19 20 11 80611 80611 80611 8061...
46,1707,36253,4.0,"fun , somewhat predictable , but very enjoyable",21 22 23 24 22 25 26 27 80611 80611 80611 8061...
76,15531,22206,5.0,great series.love hunter my favorite boyfriend...,28 29 30 31 32 33 34 11 80611 80611 80611 8061...


In [49]:
def apply_text_to_code(x):
    tokenization = nltk.word_tokenize(x.lower())
    eof_code = len(word_index_dict)  # used to filled in the blank to make up a sentence with seq_len
    index = 0
    code_int = []
    for word in tokenization:
        code_int.append(int(word_index_dict[word]))
        index += 1
    while index < sequence_len:
        code_int.append(int(eof_code))
        index += 1
    return code_int
    
for df, file_name in zip(dataframes, file_names):
    df.user_id = df.user_id.map(user_remap)
    df.product_id = df.product_id.map(product_remap)
    df['tokenized_text'] = df['review'].apply(lambda x: apply_text_to_code(x))


In [51]:
dataframes[0].head()

,user_id,product_id,rating,review,tokenized_text
6,6521.0,NaN,5.0,love this book too have read it again,"[0, 1, 2, 3, 4, 5, 6, 7, 80611, 80611, 80611, ..."
14,7557.0,NaN,5.0,i really enjoyed this book . : ),"[8, 9, 10, 1, 2, 11, 12, 13, 80611, 80611, 806..."
37,5667.0,NaN,5.0,fast paced and a load of action .,"[14, 15, 16, 17, 18, 19, 20, 11, 80611, 80611,..."
46,1208.0,NaN,4.0,"fun , somewhat predictable , but very enjoyable","[21, 22, 23, 24, 22, 25, 26, 27, 80611, 80611,..."
76,NaN,NaN,5.0,great series.love hunter my favorite boyfriend...,"[28, 29, 30, 31, 32, 33, 34, 11, 80611, 80611,..."


In [13]:
for df, file_name in zip(dataframes, file_names):
    df.to_csv(join(data_dir, file_name[:-4] + '.csv'))

In [50]:
config_dict = {}
config_dict['seq_len'] = sequence_len
config_dict['vocabulary_size'] = len(word_index_dict)
config_dict['user_num'] = len(sorted_users)
config_dict['product_num'] = len(sorted_products)

write_json(join(data_dir, 'word_index_dict.json'), word_index_dict)
write_json(join(data_dir, 'index_word_dict.json'), index_word_dict)
write_json(join(data_dir, 'config.json'), config_dict)


In [15]:
batch_size = 64
df = dataframes[0]
token_stream = df[['user_id', 'product_id', 'rating', 'tokenized_text']].values

In [16]:
dataframes[1].head()

,user_id,product_id,rating,review,tokenized_text
2,2351,64952,5.0,really like all of the joe dillard series,9 135 122 19 38 2542 8216 39 80611 80611 80611...
3,7874,42996,1.0,pretty terrible . at least it was ago,100 240 11 254 1805 6 113 199 80611 80611 8061...
14,4851,12597,5.0,excellent read ! will continue the series .,191 5 52 148 149 38 39 11 80611 80611 80611 80...
20,7535,46067,5.0,"omg , this was a great read .",234 22 1 113 17 28 5 11 80611 80611 80611 8061...
27,11019,74303,5.0,really cute . loved it start to finish,9 425 11 112 6 71 37 72 80611 80611 80611 8061...


In [17]:
num_batch = int(len(token_stream) / batch_size)
token_stream = token_stream[:num_batch * batch_size]
sequence_batches = np.split(np.array(token_stream), num_batch, axis=0)

In [19]:
sequence_batches[0][0]

array([9200, 77458, 5.0,
       '0 1 2 3 4 5 6 7 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 \n'],
      dtype=object)